Clone the github repository


In [ ]:
%cd '/content/YOLOv3/'
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import shutil
import matplotlib.pyplot as plt
from Model import Create_YOLOv3
from DataPipeLine import TrainBatchGenerator, TestBatchGenerator
from Loss import compute_loss
from Utils import*
from Config import*

Download the VOC12 dataset and save it in dataset folder

In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
shutil.unpack_archive("/content/VOCtrainval_11-May-2012", "/content/YOLOv3/dataset/", 'tar') #extraction of voc12 dataset

Training

In [ ]:
trainset = TrainBatchGenerator() #training batch generator
model = Create_YOLOv3(input_size=INPUT_SIZE, channels=3, training=True)
load_yolo_weights(model, WEIGHT_PATH) #load pretrained weights on the model
freeze(model) #this is fine tuning. only the extraction part of model will be freezed.
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6) #optimizer to use

train_lr_init=TRAIN_LR_INIT
train_lr_end=TRAIN_LR_END
steps_per_epoch = len(trainset)
global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
warmup_steps = TRAIN_WARMUP_EPOCHS * steps_per_epoch
total_steps = TRAIN_EPOCHS * steps_per_epoch

In [ ]:
def train_step(epoch, image_data, target):
    with tf.GradientTape() as tape:
        pred_result = model(image_data, training=True)
        giou_loss=conf_loss=prob_loss=0

        for i in range(3):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            loss_items = compute_loss(pred, conv, *target[i], i)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]
        total_loss = giou_loss + conf_loss + prob_loss
        
        gradients = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        tf.print("=> STEP %4d   lr: %.6f   giou_loss: %4.2f   conf_loss: %4.2f   "
                 "prob_loss: %4.2f   total_loss: %4.2f" %(global_steps, optimizer.lr.numpy(),
                                                          giou_loss, conf_loss,
                                                          prob_loss, total_loss))
        global_steps.assign_add(1)
        #learning rate will be changed as a cosine function.
        if global_steps < warmup_steps:
            lr = global_steps / warmup_steps * TRAIN_LR_INIT
        else:
            lr = TRAIN_LR_END + 0.5 * (TRAIN_LR_INIT - TRAIN_LR_END)*(
                (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi)))
        optimizer.lr.assign(lr.numpy())
for epoch in range(TRAIN_EPOCHS):
    for image_data, target in trainset:
        train_step(epoch, image_data, target)

Testing

In [ ]:
testset = TestBatchGenerator() #test batch generator
model = Create_YOLOv3(input_size=INPUT_SIZE, channels=3, training=False)
load_yolo_weights(model, WEIGHT_FILE) #load pretrained weights on the model

In [ ]:
def test_step(original_image, image_data):
    with tf.GradientTape() as tape:
        pred_bbox = model(image_data, training=False)
        pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
        pred_bbox = tf.concat(pred_bbox, axis=0)
        bbox = postprocess_boxes(pred_bbox, original_image, INPUT_SIZE, SCORE_THRESHOLD)
    bbox = nms(bbox, IOU_LOSS_THRESH, method='nms')
    image = draw_bbox(original_image, bbox)
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.show()
        

for original_image, image_data in testset:
    test_step(original_image, image_data)